In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_path = '../input/g-research-crypto-forecasting'
assets = pd.read_csv(os.path.join(data_path, 'asset_details.csv'))
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
train_df['asset_name'] = train_df.Asset_ID.map(assets.set_index('Asset_ID').Asset_Name)
train_df.head()

In [ ]:
train_df['Target'] = train_df['Target'].fillna(0)
train_df['VWAP'] = train_df['VWAP'].fillna(0)

In [ ]:
import time
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

In [ ]:
bit = train_df[train_df.Asset_ID == 1].set_index('timestamp')
bit = bit.loc[totimestamp('01/01/2021'):totimestamp('01/04/2021')]
bit.head()

In [ ]:
!pip install stockstats
from stockstats import StockDataFrame

In [ ]:
bit.drop(['Asset_ID','asset_name', 'VWAP', 'Target'], axis=1, inplace=True)
bit.rename(columns = {'Count':'amount','Open':'open','High':'high','Low':'low',
                     'Close':'close','Volume':'volume'}, inplace=True)
bit.head()

In [ ]:
stock = StockDataFrame.retype(bit)

In [ ]:
KDJ_WINDOW = 10
BOLL_WINDOW = 10
MACD_EMA_SHORT = 10
PDI_SMMA = 10
MDI_SMMA = 10
DX_SMMA = 10
ADX_EMA = 5
ADXR_EMA = 10
TRIX_EMA_WINDOW = 10
TEMA_EMA_WINDOW = 10

In [ ]:
bit['volume_delta'] = stock['volume_delta']
bit['open_-2_r'] = stock['open_-2_r']
bit['cr']= stock['cr']
bit['kdjk'] = stock['kdjk']
bit['open_2_sma'] = stock['open_2_sma']
bit['macd'] = stock['macd']
bit['boll'] = stock['boll']
bit['boll_ub'] = stock['boll_ub']
bit['boll_lb'] = stock['boll_lb']
bit['rsi_12'] = stock['rsi_12']
bit['wr_10'] = stock['wr_10']
bit['cci'] = stock['cci']
bit['tr'] = stock['tr']
bit['atr'] = stock['atr']
bit['dma'] = stock['dma']
bit['pdi'] = stock['pdi']
bit['dx'] = stock['dx']
bit['adx'] = stock['adx']
bit['adxr'] = stock['adxr']
bit['trix'] = stock['trix']
bit['tema'] = stock['tema']
bit['vr'] = stock['vr']

In [ ]:
bit.head()

In [ ]:
bit['target'] = bit.open.shift(-1) - bit.close
bit['target'] = bit.target.apply(lambda x:1 if x>0 else 0)
bit.head()

In [ ]:
bit.target.value_counts()

In [ ]:
bit = bit.dropna()

In [ ]:
X = bit.drop(['target'],axis=1).values
y = bit.target.values

In [ ]:
train_size = int(len(X)*0.70)
train, test = X[0:train_size], X[train_size:len(X)]
y_train, y_test = y[0:train_size], y[train_size:len(X)]

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression(solver='liblinear')
logistic_model.fit(train, y_train)

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred = logistic_model.predict(test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy is :{accuracy}')
precision = precision_score(y_test, y_pred, average='binary')
print(f'Precision is :{precision}')
recall = recall_score(y_test, y_pred, average='binary')
print(f'Recall is :{recall}')
confusionmatrix = confusion_matrix(y_test, y_pred)
print(confusionmatrix)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(train, y_train)

In [ ]:
y_pred = rf_model.predict(test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy is :{accuracy}')
precision = precision_score(y_test, y_pred, average='binary')
print(f'Precision is :{precision}')
recall = recall_score(y_test, y_pred, average='binary')
print(f'Recall is :{recall}')
confusionmatrix = confusion_matrix(y_test, y_pred)
print(confusionmatrix)

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(n_estimators=500, alpha=0)
xgb_model.fit(train, y_train)

In [ ]:
y_pred = xgb_model.predict(test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy is :{accuracy}')
precision = precision_score(y_test, y_pred, average='binary')
print(f'Precision is :{precision}')
recall = recall_score(y_test, y_pred, average='binary')
print(f'Recall is :{recall}')
confusionmatrix = confusion_matrix(y_test, y_pred)
print(confusionmatrix)